##NYS Hospitalizations Records - data preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [100]:
def transform_column_names(dataframe):
    """Convert column names in a dataframe to lower case and join multi-word names to a single-word name"""
    
    dataframe.columns = [x.lower() for x in dataframe.columns]
    dataframe.columns = [x.replace(" ","") for x in dataframe.columns]
    return dataframe.columns

In [104]:
def series_to_dataframe(values,index_,value_):
    """Converting pandas series to dataframe with two columns:
    col1 - series index
    col2 - series value
    index_ and value_ are strings (column names of returned dataframe)"""
    
    col1 = values.index.tolist()
    s1 = pd.Series(col1)

    col2 = []
    for value in values:
        col2.append(value)
    s2 = pd.Series(col2)

    df = pd.concat([s1,s2],axis=1)
    df.columns = [index_,value_]
    return df

In [103]:
def dataframe_subset(dataframe,feature,selection):
    """Selecting a subset of dataframe with feature set to selection criteria and reindexing a dataframe
    feature - string
    selection - variable/feature type"""
    
    df = dataframe[dataframe[feature] == selection].reset_index(drop = True)
    return df

In [109]:
def NaNprob_per_col(dataframe):
    """Find probability of NaN values in each column of a dataframe
    return a 2-column dataframe: col1 - feature, col2 - NaN probability """
    
    nv = dataframe.isnull().sum()/len(dataframe)
    NaN_val = series_to_dataframe(nv,'feature','NaN_perc')
    return NaN_val

In [127]:
def remove_columns(dataframe,features):
    """Deletes columns in dataframe
    features - list of column names that need to be removed from a dataframe"""
    for feature in features:
        if feature in dataframe:
            del dataframe[feature]
        else:
            print feature + ' not in ' + dataframe
    return dataframe  

In [106]:
def dropna_(dataframe):
    """Dropping NaNs"""
    
    dataframe = dataframe.dropna()
    return dataframe

In [13]:
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',300)

In [9]:
!ls -l *.csv

-rw-r--r--@ 1 iwonapawelczak  staff  1008218962 Mar 11  2015 Hospital_Inpatient_Discharges__SPARCS_De-Identified___2011.csv
-rw-r--r--@ 1 iwonapawelczak  staff   913524471 Mar 11  2015 Hospital_Inpatient_Discharges__SPARCS_De-Identified___2013.csv
-rw-r--r--  1 iwonapawelczak  staff     4969037 Jun 23  2015 NY_CesareanSection_2011.csv


In [12]:
df = pd.read_csv("Hospital_Inpatient_Discharges__SPARCS_De-Identified___2011.csv",low_memory = False)

In [14]:
df.head()

,Health Service Area,Hospital County,Operating Certificate Number,Facility ID,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,Admit Day of Week,Type of Admission,Patient Disposition,Discharge Year,Discharge Day of Week,CCS Diagnosis Code,CCS Diagnosis Description,CCS Procedure Code,CCS Procedure Description,APR DRG Code,APR DRG Description,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Source of Payment 1,Source of Payment 2,Source of Payment 3,Attending Provider License Number,Operating Provider License Number,Other Provider License Number,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
0,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,30 to 49,123,M,White,Not Span/Hispanic,3,FRI,Emergency,Home or Self Care,2011,MON,50,DIABETES W/COMPL,58,HEMODIALYSIS,420,DIABETES,10,"Endocrine, Nutritional and Metabolic Diseases ...",4,Extreme,Major,Medical,Medicare,Self-Pay,NaN,236885,236885,NaN,0,N,Y,$39826.76,$10488.51
1,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,0 to 17,120,M,White,Not Span/Hispanic,1,FRI,Emergency,Home or Self Care,2011,SAT,8,OTHER INFECTIONS,162,OT OR THER PRC ON JOINT,710,INFECTIOUS & PARASITIC DISEASES INCLUDING HIV ...,18,"Infectious and Parasitic Diseases, Systemic or...",2,Moderate,Minor,Surgical,Insurance Company,Self-Pay,NaN,244921,252318,NaN,0,N,Y,$9553.33,$2528.70
2,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,18 to 29,121,F,White,Not Span/Hispanic,2,FRI,Emergency,Home or Self Care,2011,SUN,149,BILIARY TRACT DISEASE,84,CHOLECYSTECTOMY/EXPLOR,263,LAPAROSCOPIC CHOLECYSTECTOMY,7,Diseases and Disorders of the Hepatobiliary Sy...,2,Moderate,Minor,Surgical,Insurance Company,Self-Pay,NaN,204436,204436,NaN,0,N,Y,$21694.42,$6705.37
3,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,0 to 17,122,F,Black/African American,Not Span/Hispanic,1,FRI,Emergency,Expired,2011,FRI,238,SURGCL/MED CARE COMPL,216,RESP INTUB/MECH VENTIL,132,BPD & OTH CHRONIC RESPIRATORY DISEASES ARISING...,4,Diseases and Disorders of the Respiratory System,4,Extreme,Major,Medical,Medicaid,Self-Pay,NaN,251930,251040,NaN,0,N,Y,$14000.54,$3890.69
4,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,18 to 29,122,F,Black/African American,Not Span/Hispanic,3,FRI,Emergency,Psychiatric Hospital or Unit of Hosp,2011,MON,241,PSYCHOTROPIC POISONING,0,NO PROC,812,POISONING OF MEDICINAL AGENTS,21,"Poisonings, Toxic Effects, Other Injuries and ...",2,Moderate,Minor,Medical,Insurance Company,Medicaid,Self-Pay,3814,NaN,NaN,0,N,Y,$12260.63,$4466.34


In [27]:
# All hospitalizations
print df.shape
print df.dtypes
print df.columns

(2584770, 39)
Health Service Area                     object
Hospital County                         object
Operating Certificate Number           float64
Facility ID                            float64
Facility Name                           object
Age Group                               object
Zip Code - 3 digits                     object
Gender                                  object
Race                                    object
Ethnicity                               object
Length of Stay                          object
Admit Day of Week                       object
Type of Admission                       object
Patient Disposition                     object
Discharge Year                           int64
Discharge Day of Week                   object
CCS Diagnosis Code                       int64
CCS Diagnosis Description               object
CCS Procedure Code                       int64
CCS Procedure Description               object
APR DRG Code                             int64

In [30]:
transform_column_names(df)

Index([u'healthservicearea', u'hospitalcounty', u'operatingcertificatenumber', u'facilityid', u'facilityname', u'agegroup', u'zipcode-3digits', u'gender', u'race', u'ethnicity', u'lengthofstay', u'admitdayofweek', u'typeofadmission', u'patientdisposition', u'dischargeyear', u'dischargedayofweek', u'ccsdiagnosiscode', u'ccsdiagnosisdescription', u'ccsprocedurecode', u'ccsproceduredescription', u'aprdrgcode', u'aprdrgdescription', u'aprmdccode', u'aprmdcdescription', u'aprseverityofillnesscode', u'aprseverityofillnessdescription', u'aprriskofmortality', u'aprmedicalsurgicaldescription', u'sourceofpayment1', u'sourceofpayment2', u'sourceofpayment3', u'attendingproviderlicensenumber', u'operatingproviderlicensenumber', u'otherproviderlicensenumber', u'birthweight', u'abortioneditindicator', u'emergencydepartmentindicator', u'totalcharges', u'totalcosts'], dtype='object')

In [112]:
CS = dataframe_subset(df,'ccsproceduredescription','CESAREAN SECTION')

In [113]:
# Check for data dimentions/stats, types, and column names
print "Data shape: ", CS.shape
print "Data types: ", CS.dtypes
print "Columns: ", CS.columns

Data shape:  (79217, 39)
Data types:  healthservicearea                   object
hospitalcounty                      object
operatingcertificatenumber         float64
facilityid                         float64
facilityname                        object
agegroup                            object
zipcode-3digits                     object
gender                              object
race                                object
ethnicity                           object
lengthofstay                        object
admitdayofweek                      object
typeofadmission                     object
patientdisposition                  object
dischargeyear                        int64
dischargedayofweek                  object
ccsdiagnosiscode                     int64
ccsdiagnosisdescription             object
ccsprocedurecode                     int64
ccsproceduredescription             object
aprdrgcode                           int64
aprdrgdescription                   object
aprmdccode      

In [114]:
CS.head()

,healthservicearea,hospitalcounty,operatingcertificatenumber,facilityid,facilityname,agegroup,zipcode-3digits,gender,race,ethnicity,lengthofstay,admitdayofweek,typeofadmission,patientdisposition,dischargeyear,dischargedayofweek,ccsdiagnosiscode,ccsdiagnosisdescription,ccsprocedurecode,ccsproceduredescription,aprdrgcode,aprdrgdescription,aprmdccode,aprmdcdescription,aprseverityofillnesscode,aprseverityofillnessdescription,aprriskofmortality,aprmedicalsurgicaldescription,sourceofpayment1,sourceofpayment2,sourceofpayment3,attendingproviderlicensenumber,operatingproviderlicensenumber,otherproviderlicensenumber,birthweight,abortioneditindicator,emergencydepartmentindicator,totalcharges,totalcosts
0,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,30 to 49,120,F,White,Not Span/Hispanic,8,SAT,Emergency,Home or Self Care,2011,SUN,190,FETAL DISTRESS,134,CESAREAN SECTION,540,CESAREAN DELIVERY,14,"Pregnancy, Childbirth and the Puerperium",3,Major,Minor,Surgical,Insurance Company,Medicaid,Self-Pay,114221,171241,114221,0,N,Y,$25584.21,$11622.46
1,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,18 to 29,128,F,White,Not Span/Hispanic,4,SAT,Emergency,Home or Self Care,2011,WED,182,PREGNANCY HEMORRHAG,134,CESAREAN SECTION,540,CESAREAN DELIVERY,14,"Pregnancy, Childbirth and the Puerperium",2,Moderate,Minor,Surgical,Insurance Company,Insurance Company,Self-Pay,196010,254742,254742,0,N,Y,$41648.20,$11220.69
2,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,30 to 49,OOS,F,White,Not Span/Hispanic,18,SUN,Emergency,Inpatient Rehabilitation Facility,2011,THU,195,OTH COMP BIRTH/PUERPRM,134,CESAREAN SECTION,540,CESAREAN DELIVERY,14,"Pregnancy, Childbirth and the Puerperium",4,Extreme,Major,Surgical,Insurance Company,Self-Pay,NaN,216291,149551,NaN,0,N,Y,$143534.01,$50545.41
3,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,30 to 49,120,F,White,Not Span/Hispanic,17,WED,Emergency,Home w/ Home Health Services,2011,SAT,195,OTH COMP BIRTH/PUERPRM,134,CESAREAN SECTION,540,CESAREAN DELIVERY,14,"Pregnancy, Childbirth and the Puerperium",3,Major,Major,Surgical,Insurance Company,Medicaid,Self-Pay,149551,149551,149551,0,N,Y,$128973.98,$48055.58
4,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,30 to 49,122,F,Black/African American,Not Span/Hispanic,11,FRI,Emergency,Home or Self Care,2011,TUE,182,PREGNANCY HEMORRHAG,134,CESAREAN SECTION,540,CESAREAN DELIVERY,14,"Pregnancy, Childbirth and the Puerperium",3,Major,Moderate,Surgical,Insurance Company,Medicaid,Self-Pay,148296,148296,148296,0,N,Y,$76803.96,$27072.54


In [115]:
CS['lengthofstay'] = CS.lengthofstay.str.replace('+', '').astype(float)

In [116]:
CS['ccsdiagnosiscode'] = df['ccsdiagnosiscode'].astype(float)

In [117]:
CS['totalcharges'] = CS['totalcharges'].str.replace(r'$','').astype(float)
CS['totalcosts'] = CS['totalcosts'].str.replace(r'$','').astype(float)

In [118]:
nan_values = CS.isnull().sum()

In [119]:
nan_values

healthservicearea                      9
hospitalcounty                         9
operatingcertificatenumber             9
facilityid                             9
facilityname                           0
agegroup                               0
zipcode-3digits                      125
gender                                 0
race                                   0
ethnicity                              0
lengthofstay                           0
admitdayofweek                         9
typeofadmission                        0
patientdisposition                     1
dischargeyear                          0
dischargedayofweek                     9
ccsdiagnosiscode                       0
ccsdiagnosisdescription                0
ccsprocedurecode                       0
ccsproceduredescription                0
aprdrgcode                             0
aprdrgdescription                      0
aprmdccode                             0
aprmdcdescription                      0
aprseverityofill

In [121]:
# Check for columns that contain large numbers of missing values - set criteria for column removal based on NaNs
NaN_prob = NaNprob_per_col(CS)

In [122]:
NaN_prob

,feature,NaN_perc
0,healthservicearea,0.000114
1,hospitalcounty,0.000114
2,operatingcertificatenumber,0.000114
3,facilityid,0.000114
4,facilityname,0.000000
5,agegroup,0.000000
6,zipcode-3digits,0.001578
7,gender,0.000000
8,race,0.000000
9,ethnicity,0.000000


In [123]:
# remove empty columns which have large numbers of NaNs
# it would suffice to remove if 30% of data is missing but zip-code has 0.16% NaNs and it is collinear with other 
# variables e.g., hospitals so can be removed here as well
features_to_remove = NaN_prob[NaN_prob['NaN_perc'] >= 0.001]

In [124]:
items = features_to_remove['feature'].tolist()
print 'List of features to be removed from CS dataframe based on NaNs condition:'
print items

List of features to be removed from CS dataframe based on NaNs condition:
['zipcode-3digits', 'sourceofpayment2', 'sourceofpayment3', 'otherproviderlicensenumber']


In [125]:
# Number of columns before removal
CS.shape

(79217, 39)

In [128]:
# Removing columns which are mostly empty + zip
CS = remove_columns(CS,items)

In [129]:
# After empty columns removal
CS.shape

(79217, 35)

In [130]:
CS = dropna_(CS)

In [131]:
# After removing NaN values - data for further analysis
CS.shape

(79204, 35)

In [132]:
CS.to_csv("NY_CesareanSection_2011_70kevents.csv")